# TP n°3: Suivi Multi-Objet
**Attention:** Ce notebook contient un certain nombre de commande devant être exécuté dans un envrionnement linux. Si ça ne marche pas, nous vons conseillons d'exécuter le notebook sur Google Colaboratory. Nous avons obtenue nos résultats en exécutant notre code sur cette plateforme. \
Voici le lien vers le notebook sur colab: https://colab.research.google.com/drive/1Ej3rr2GBQDbI89I1wsjqaNRdqXkDOdMe?usp=sharing \
\
**Membres:**
- Guillaume Girouard 1954899
- Pierre-Emmanuel Rebours 2165286

On importe les librairies nécessaires

In [1]:
!pip install ocsort

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ocsort: filename=ocsort-0.0.2-py3-none-any.whl size=22536 sha256=9a3d60d1ca2fb6ec00bd98a5e18d866fdf23110f7beefc54831d3a8716d3ba11
  Stored in directory: /root/.cache/pip/wheels/e2/05/c3/3caee42b721e6d2c774ce5b3476682ffc6c17c1b1e2d644ba1
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110473 sha256=119263a965495487dd06b82869a5820bba9c42dd23c5572e8a37c7d3d435e124
  Stored in directory: /root/.ca

In [2]:
from ocsort.ocsort import OCSort as OCSort

In [3]:
#!git clone https://github.com/noahcao/OC_SORT.git
!git clone https://github.com/JonathonLuiten/TrackEval.git

Cloning into 'TrackEval'...
remote: Enumerating objects: 924, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 924 (delta 34), reused 44 (delta 25), pack-reused 861
Receiving objects: 100% (924/924), 423.37 KiB | 3.68 MiB/s, done.
Resolving deltas: 100% (597/597), done.


In [4]:
!wget https://omnomnom.vision.rwth-aachen.de/data/TrackEval/data.zip
!unzip data.zip -d TrackEval

--2023-04-15 18:41:49--  https://omnomnom.vision.rwth-aachen.de/data/TrackEval/data.zip
Resolving omnomnom.vision.rwth-aachen.de (omnomnom.vision.rwth-aachen.de)... 137.226.34.132
Connecting to omnomnom.vision.rwth-aachen.de (omnomnom.vision.rwth-aachen.de)|137.226.34.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149957160 (143M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 143.01M  24.7MB/s    in 6.4s    

2023-04-15 18:41:56 (22.3 MB/s) - ‘data.zip’ saved [149957160/149957160]

Archive:  data.zip
   creating: TrackEval/data/
   creating: TrackEval/data/gt/
   creating: TrackEval/data/gt/bdd100k/
   creating: TrackEval/data/gt/bdd100k/bdd100k_val/
  inflating: TrackEval/data/gt/bdd100k/bdd100k_val/b1c66a42-6f7d68ca.json  
  inflating: TrackEval/data/gt/bdd100k/bdd100k_val/b1c81faa-3df17267.json  
  inflating: TrackEval/data/gt/bdd100k/bdd100k_val/b1c81faa-c80764c5.json  
  inflating: TrackEval/data/gt/bdd100k/b

In [5]:
!pip install natsort

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import os, sys, time, datetime, random
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import cv2
import numpy as np

#from OC_SORT.trackers.ocsort_tracker.ocsort import OCSort
from dataloader import dataloader

In [7]:
import tensorflow as tf

In [8]:
!pip install filterpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Récupération des résultats pour MOT17

Les commandes suivantes permettent d'installer les données du dataset MOT17 mais aussi de préparer l'utilisation de TrackEval.

In [9]:
!wget https://motchallenge.net/data/MOT17.zip

--2023-04-15 18:43:03--  https://motchallenge.net/data/MOT17.zip
Resolving motchallenge.net (motchallenge.net)... 131.159.19.34, 2a09:80c0:18::1034
Connecting to motchallenge.net (motchallenge.net)|131.159.19.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5860214001 (5.5G) [application/zip]
Saving to: ‘MOT17.zip’

MOT17.zip           100%[===================>]   5.46G  17.5MB/s    in 5m 44s  

2023-04-15 18:48:47 (16.2 MB/s) - ‘MOT17.zip’ saved [5860214001/5860214001]



In [10]:
!unzip MOT17.zip

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: MOT17/train/MOT17-04-DPM/img1/000455.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000333.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/001039.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000327.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000441.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000496.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000482.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000509.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000521.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000247.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000253.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000535.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000284.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000290.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000079.jpg  
  inflating: MOT17/train/MOT17-04-DPM/img1/000045.jpg  
  inflating: MOT17/train/M

In [11]:
!mkdir ./TrackEval/data/trackers/mot_challenge/MOT17_tp-train

In [12]:
!mkdir ./TrackEval/data/trackers/mot_challenge/MOT17_tp-train/OC-SORT

In [13]:
!mkdir ./TrackEval/data/trackers/mot_challenge/MOT17_tp-train/OC-SORT/data

In [15]:
!mkdir ./TrackEval/data/gt/mot_challenge/MOT17_tp-train

mkdir: cannot create directory ‘./TrackEval/data/gt/mot_challenge/MOT17_tp-train’: File exists


In [14]:
!cp -R ./TrackEval/data/gt/mot_challenge/MOT17-train/MOT17-11-DPM ./TrackEval/data/gt/mot_challenge/MOT17_tp-train

In [16]:
!echo -e 'name\nMOT17-02-DPM\nMOT17-04-DPM\nMOT17-05-DPM\nMOT17-09-DPM\nMOT17-10-DPM\nMOT17-11-DPM\nMOT17-13-DPM' > './TrackEval/data/gt/mot_challenge/seqmaps/MOT17_tp-train.txt'

In [17]:
!cp './TrackEval/data/gt/mot_challenge/seqmaps/MOT17_tp-train.txt' './TrackEval/data/gt/mot_challenge/seqmaps/MOT17_tp-all.txt'

In [18]:
!cp './TrackEval/data/gt/mot_challenge/seqmaps/MOT17_tp-train.txt' './TrackEval/data/gt/mot_challenge/seqmaps/MOT17_tp-test.txt'

In [19]:
nb_frame=dict({"MOT17-02-DPM":600,"MOT17-04-DPM":1050,"MOT17-05-DPM":837,
               "MOT17-09-DPM":525,"MOT17-10-DPM":654,"MOT17-11-DPM":900,
               "MOT17-13-DPM":750})

In [20]:
!cp -R './TrackEval/data/gt/mot_challenge/MOT17-train/MOT17-02-DPM' './TrackEval/data/gt/mot_challenge/MOT17_tp-train/'
!cp -R './TrackEval/data/gt/mot_challenge/MOT17-train/MOT17-04-DPM' './TrackEval/data/gt/mot_challenge/MOT17_tp-train/'

In [21]:
!cp -R './TrackEval/data/gt/mot_challenge/MOT17-train/MOT17-05-DPM' './TrackEval/data/gt/mot_challenge/MOT17_tp-train/'
!cp -R './TrackEval/data/gt/mot_challenge/MOT17-train/MOT17-09-DPM' './TrackEval/data/gt/mot_challenge/MOT17_tp-train/'
!cp -R './TrackEval/data/gt/mot_challenge/MOT17-train/MOT17-10-DPM' './TrackEval/data/gt/mot_challenge/MOT17_tp-train/'
!cp -R './TrackEval/data/gt/mot_challenge/MOT17-train/MOT17-11-DPM' './TrackEval/data/gt/mot_challenge/MOT17_tp-train/'
!cp -R './TrackEval/data/gt/mot_challenge/MOT17-train/MOT17-13-DPM' './TrackEval/data/gt/mot_challenge/MOT17_tp-train/'

On importe le modèle utilisé pour la detection: Yolov5x

In [22]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5x', pretrained=True)

/usr/local/lib/python3.9/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-4-15 Python-3.9.16 torch-2.0.0+cu118 CPU



requirements: /root/.cache/torch/hub/requirements.txt not found, check failed.


100%|██████████| 166M/166M [00:01<00:00, 152MB/s]

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients
Adding AutoShape... 


In [25]:
def get_result(dl,nb_frame):
  """
    fonction pour appliquer notre méthode sur un dataframe et enregistrer les résultats dans un fichier .txt
    Args:
        dl: dataloader important un dataset
        nb_frame: nombre de frame du dataset récupérer par le dataloader
  """
  #Il faut réinitialiser le traceur à chaque nouveau dataset:
  tracker = OCSort()
  tracker.det_thresh=0.5
  for i in range(nb_frame):
    image=dl.load_input(i)
    img_size=np.shape(image)

    #Détection
    results = model(image)
    res=results.pandas().xyxy[0][['xmin','ymin','xmax','ymax','confidence','class']]
    cup_results=res[res['class']==0]
    np_res=cup_results.to_numpy()
    tf_res=tf.constant(np_res)
    #suivi
    online_targets = tracker.update(tf_res,_)
    dl.load_result(i+1,online_targets,benchmark_type='MOT')
  dl.f.close()


Récupération des résultats pour tous les dataframe de MOT17_tp

In [26]:
f=open('./TrackEval/data/gt/mot_challenge/seqmaps/MOT17_tp-train.txt')
for i,row in enumerate(f):
  if i>=1:
    row=row[:-1]
    dl=dataloader("./MOT17/train/"+row+"/img1/","./TrackEval/data/trackers/mot_challenge/MOT17_tp-train/OC-SORT/data/"+row+".txt")
    get_result(dl,nb_frame[row])
    print("Résultat du dataset "+row+" disponible")
f.close()

Résultat du dataset MOT17-02-DPM disponible
Résultat du dataset MOT17-04-DPM disponible
Résultat du dataset MOT17-05-DPM disponible
Résultat du dataset MOT17-09-DPM disponible
Résultat du dataset MOT17-10-DPM disponible
Résultat du dataset MOT17-11-DPM disponible
Résultat du dataset MOT17-13-DPM disponible


Calcul des performances

In [27]:
!python TrackEval/scripts/run_mot_challenge.py --BENCHMARK MOT17_tp --SPLIT_TO_EVAL train --TRACKERS_TO_EVAL OC-SORT --METRICS HOTA --USE_PARALLEL False --NUM_PARALLEL_CORES 1


Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 1                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : /content/TrackEval/error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : False                         
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

MotChallenge2DBox Config:
PRINT_CONFIG         : True                          
GT_FOLDER            : /content/TrackEval/data/gt/mot_challenge/
TRACKERS_FOLDER      : /content/TrackEval/data/trackers/mot_challenge/
OUTPUT_FOL

In [28]:
!python TrackEval/scripts/run_mot_challenge.py --BENCHMARK MOT17_tp --SPLIT_TO_EVAL train --TRACKERS_TO_EVAL OC-SORT --METRICS HOTA --USE_PARALLEL False --NUM_PARALLEL_CORES 1


Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 1                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : /content/TrackEval/error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : False                         
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

MotChallenge2DBox Config:
PRINT_CONFIG         : True                          
GT_FOLDER            : /content/TrackEval/data/gt/mot_challenge/
TRACKERS_FOLDER      : /content/TrackEval/data/trackers/mot_challenge/
OUTPUT_FOL

### Récupération des résultats pour la vidéo du tp

Importation du modèle Yolo5vx

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5x', pretrained=True)

/usr/local/lib/python3.9/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-4-12 Python-3.9.16 torch-2.0.0+cu118 CPU



[Errno 2] No such file or directory: '/usr/local/lib/python3.9/dist-packages/scipy-1.10.1.dist-info/METADATA'


100%|██████████| 166M/166M [00:01<00:00, 88.2MB/s]

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients
Adding AutoShape... 


Récupération des résultats

In [ ]:
dl=dataloader("./frames/","./results.txt")
nb_frame=len(dl.input_data)
tracker = OCSort()
tracker.det_thresh=0.5
for i in range(nb_frame):
  image=dl.load_input(i)
  img_size=np.shape(image)
  
  #détection
  results = model(image)
  res=results.pandas().xyxy[0][['xmin','ymin','xmax','ymax','confidence','class']]
  cup_results=res[res['class']==41]
  np_res=cup_results.to_numpy()
  tf_res=tf.constant(np_res)
  #suivi
  online_targets = tracker.update(tf_res,_)#,img_size,img_size)
  dl.load_result(i,online_targets)
dl.f.close()